# 데이터 가져오는 코드
- 샘플 데이터 읽기는 `(xtrain,ytrain),(xtest,ytest) = keras.datasets.fashion_mnist.load_data()`, (xtrain,ytrain),(xtest,ytest)형태로 불러온다.  

# 케라스 모델링의 기본 플로우
- 모델 클래스 만들기 (또는 그냥 케라스 층쌓아서 만들기)
- 모델에 loss, optimizer, metric 정보를 넣어서 compile하기
- 학습 데이터로 모델 학습 (fit)하기
- 테스트 데이터로 모델 검증 (evaluate)하기
- 새로운 인풋 데이터로 predict하기

# 모델 만드는 기본 틀

- 시작  
`keras.backend.clear_session()`  
`np.random.seed(42)`  
`tf.random.set_seed(42)`  

- 층 쌓기  
`model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])`  

- 함수형 모델 만들기  
`input_A = keras.layers.Input(shape=[5], name="wide_input")`  
`input_B = keras.layers.Input(shape=[6], name="deep_input")`  
`hidden1 = keras.layers.Dense(30, activation="relu")(input_B)`  
`hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)`  
`concat = keras.layers.concatenate([input_A, hidden2])`  
`output = keras.layers.Dense(1, name="main_output")(concat)`  
`aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)`  
`model = keras.models.Model(inputs=[input_A, input_B], outputs=[output, aux_output])`  
`keras.utils.plot_model(model, " 함수형api.png", show_shapes=True)`  

- 컴파일 (loss, optimizer, metrics)  
`model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])`

- 모델 결과 확인  
`model.summary()`  
`keras.utils.plot_model(model, "my_fashion_mnist_model.png", show_shapes=True)`

- 학습 fit  
`history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))`  
`pd.DataFrame(history.history).plot(figsize=(8, 5))`

- 모델 검증 evaluate  
`model.evaluate(X_test, y_test)`  

- 예측 predict  
`y_proba = model.predict(X_new)`  

- 모델 저장  
`model.save("my_keras_model.h5")`  
`model.save_weights("my_keras_weights.ckpt")`  
`model.load_weights("my_keras_weights.ckpt")`  
`model = keras.models.load_model("my_keras_model.h5")`  

- 텐서보드  
`import os`  
`def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(os.path.join(os.curdir, "my_logs"), run_id)`  
`run_logdir = get_run_logdir()`  
`tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)`  
`history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[tensorboard_cb])`

# 사용자 정의 모델
- keras.models.Model을 상속받는다.
- init에서는 레이어를 정의한다. 
- call(self,inputs)에서는 input및 레이어를 연결하여 output을 도출하는 함수를 짠다.

In [12]:
from tensorflow.keras.utils import plot_model
class WideAndDeepModel(keras.models.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
        
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

model = WideAndDeepModel(30, activation="relu")

# 더 쉽게 클래스를 만드는 방법 (함수형 모델)
- keras.models.Model을 상속받지 않는다. 
- init내부에서 레이어들을 정의, 연결
- keras.Model(inputs레이어,outputs레이어)로 모델 정의
- get_model 함수를 정의하여 모델 리턴하기

In [18]:
from tensorflow import keras

class WideAndDeepModelFunc:
    def __init__(self, input_A_len, input_B_len, units=30, activation='relu', **kwargs):
        
        input_A=keras.layers.Input(shape=(input_A_len,))
        input_B=keras.layers.Input(shape=(input_B_len,))
        
        hidden1 = keras.layers.Dense(units, activation=activation)(input_B)
        hidden2 = keras.layers.Dense(units, activation=activation)(hidden1)
        concat = keras.layers.concatenate([input_A, input_B])
        main_output = keras.layers.Dense(1)(concat)
        aux_output = keras.layers.Dense(1)(hidden2)
        
        self.model = keras.models.Model([input_A,input_B], [main_output,aux_output])
    
    def get_model(self):
        return self.model

input_A_len, input_B_len = 32, 32
model = WideAndDeepModelFunc(input_A_len, input_B_len).get_model()
#model.fit([input_A,input_B])

# train 데이터 외에 다른 데이터가 개입되는 모델

In [ ]:
class Modelname(tf.keras.Model):
    def __init__(
        self,
        data,
        units,
        *args,
        **kwargs,
    ):
        super(Modelname, self).__init__(*args, **kwargs)
        
        self.data = data
        
        self.custom_layer = keras.layers.Dense(units, activation=activation)
        self.output_layer = keras.layers.Dense(1)

    def call(self, x_train):
        
        x = self.customlayer(self.data)
        embeddings = tf.gather(x, x_train)
        return self.output_layer(embeddings)
    
model = Modelname(
    data=data,
    units=units
)

# Compile the model.
model.compile(
    optimizer=keras.optimizers.Adam(0.01),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)

# Fit the model.
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=100,
    batch_size=256,
    validation_split=0.15,
)
        

# 자주 쓰는 케라스 레이어
- `keras.layers.Input(shape=(x,x),dtype='xx')`
- `keras.layers.Embedding(input_dim, output_dim, input_length=none)(layer)`
- `keras.layers.Flatten()(layer)`
- `keras.layers.Multiply()([layer1,layer2])`: element-wise한 곱셈
- `keras.Concatenate()([layer1,layer2])`
- `keras.Dense(units, activation='relu')(layer)`
- `keras.Dropout(rate)(layer)`
- `keras.layers.BatchNormalization()(layer)`
- `keras.layers.StringLookup(vocabulary)` vocabulary=['str1','str2','str3'] 을 쓰면 1,2,3으로 변환됨
- `keras.layers.Normalization(axis=1)` layer.adapt(data)로 평균, 표준편차를 뽑아내고, layer(data)로 적용함.
- `keras.layers.Embedding()`

# Loss, Optimizer, Metrics

## Loss
- `loss: mean_squared_error, binary_crossentropy, categorical_crossentropy, sparse_categorical_crossentropy`

<img src = "images/loss.png">

## Optimizer
- `optimizer: SGD, Adagrad, RMSprop, Adam, Adamax, Nadam 등` 
- `optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)`
- `optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)`
- `optimizer = keras.optimizers.Adagrad(learning_rate=0.001)`
- `optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)`
- `optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)`
- `optimizer = keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999)`
- `optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)`

## Metrics
- loss: 손실함수. 훈련셋과 연관. 훈련에 사용. 
- metric: 평가지표. 검증셋과 연관. 훈련 과정을 모니터링하는데 사용. 
- `metric: AUC, accuracy, precision, recall, binary_accuracy, categorical_accuracy, sparse_categorical_accuracy, top_k_categorical_accuracy, sparse_top_k_categorical_accuracy, 그 외 loss에 포함된 모든 지표`

# Transfer 모델링

### transfer 대상이 되는 모델
- `model_A = keras.models.load_model("my_model_A.h5")`

### 서로 영향받는 레이어를 공유하는 방식
- `model_B_on_A = keras.models.Sequential(model_A.layers[:-1])`
- `model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))`

### 서로 영향받지 않게 레이어를 복사해서 공유하는 방식
- `model_A_clone = keras.models.clone_model(model_A)#모델복사 keras.models.clone_model(model_A)`
- `model_A_clone.set_weights(model_A.get_weights())#가중치복사 .set_weights(model_A.get_weights())` 
- `model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])#레이어 공유`
- `model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))#마지막 레이어 설정`

### 처음n번의 epoch동안 재사용된 레이어의 학습 상태를 동결한다.--> 4ephoch동안 훈련

In [ ]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

### 동결된 레이어의 학습 상태를 풀고, 남은 16epoch동안 훈련한다. 

In [ ]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))

In [3]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [9]:
# [name for name in dir(keras.optimizers) if not name.startswith("_")]